In [8]:
from assignee import *
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from tqdm import tqdm
engine = establish_connection()

## 01 - Sample

In [ ]:
sample_mentions(750)

## 02 - Populate Sample

In [ ]:
with engine.connect() as connection:
    populate_sample(connection=connection, sample_path="data/01 - sample.txt", output_path="data/02 - sample_with_data.csv")

## 03 - Segement Sample
Take the populated sample from Step 02 and split into `n` different files where `n` is the number of hand labelers. Let's assume `n=3` for now.

In [ ]:
segment_sample()

# 04 - Match Organizations
This step is done using streamlit.

# 05 - Extraction

In [19]:
assignee_disambiguation_IDs = ["6e8e8a6c-a5ef-485c-adf7-cb0e654e8e8b", "3c02a0b9-06d5-4486-850a-93680e052484"]

In [20]:
id_list = '("' + '","'.join(assignee_disambiguation_IDs) + '")'
query = f"SELECT * FROM algorithms_assignee_labeling.assignee a WHERE a.disambiguated_assignee_id IN {id_list}"
with engine.connect() as connection:
    result = connection.execute(text(query)).fetchall()
df = pd.DataFrame(result).drop_duplicates()

In [21]:
# Create a new Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active

# Loop through mention_id groups and sort values
for group, data in tqdm(df.groupby(['patent_id', 'assignee_sequence']), desc="Assignee Mention Loop"):
    df_temp = data.sort_values(by=["inventor_id", "cpc_subgroup_id"], inplace=False)

    # Determine indexing values and add rows to worksheets
    index_start = ws.max_row + 1
    index_end = index_start + len(df_temp.index) - 1
    for row in dataframe_to_rows(df_temp, index=False, header=(index_start == 2)):
        ws.append(row)

    merge_cells(ws, index_start, index_end, len(df.columns))

Assignee Mention Loop: 100%|██████████| 2/2 [00:00<00:00, 27.66it/s]


# X - Testing for streamlit

In [5]:
query = "SELECT * FROM algorithms_assignee_labeling.assignee WHERE assignee.organization='Lutron Electronics Co., Inc.'"
with engine.connect() as connection:
    results = connection.execute(text(query)).fetchall()
pd.DataFrame(results)

[('D477289', 'Switch', None, datetime.date(2003, 7, 15), 'design', 'S1', '63f4ddc7-1e20-4ea4-ac21-48b20d6c634d', 'Lutron Electronics Co., Inc.', None, None, 0, 2, 'Coopersburg', 'PA', 'US', '2001/29151527', '29/151527', 'US', '29151527', '29151527', '29', datetime.date(2001, 11, 13), datetime.date(2003, 7, 15), None, None, None, None, None, None, None, None, None, 'fl:no_ln:mayo-2', 0, 'Noel', 'Mayo', 'Philadelphia', 'PA', 'US'),
 ('D477289', 'Switch', None, datetime.date(2003, 7, 15), 'design', 'S1', '63f4ddc7-1e20-4ea4-ac21-48b20d6c634d', 'Lutron Electronics Co., Inc.', None, None, 0, 2, 'Coopersburg', 'PA', 'US', '2001/29151527', '29/151527', 'US', '29151527', '29151527', '29', datetime.date(2001, 11, 13), datetime.date(2003, 7, 15), None, None, None, None, None, None, None, None, None, 'fl:ja_ln:adams-10', 1, 'Jason O.', 'Adams', 'Emmaus', 'PA', 'US'),
 ('D477289', 'Switch', None, datetime.date(2003, 7, 15), 'design', 'S1', '63f4ddc7-1e20-4ea4-ac21-48b20d6c634d', 'Lutron Electronic